# Ituran Second Response charging points SoH esitmation

In this notebook we will handle the processed time series data to compute the SoH from the charging points (like we did with Watea).  
This would corresponds to the result/soh_estimation step in our pipeline.  

## Setup
Please run the `ituran_second_response_tss_EDA.ipynb` notebook before running this one.  

### Import

In [ ]:
import plotly.express as px

from core.pandas_utils import *
from core.plt_utils import plt_3d_df

### Data Extraction

We will simply load the processed time series data computed in the previous notebook.

In [ ]:
tss = pd.read_parquet("./data_cache/ituran_second_response_tss.parquet")

## SoH estimation  
To estimate the SoH, we will use the charging points' energy_added.  
In this context a charging point is an statistical description of a period of charging from one soc to the next.  
We consider the energy added to be the energy that the battery required to gain that soc.  
We will estimate/express the SoH of the battery as the energy required by the battery to gain a certain soc divided by the energy required by a 100%SoH battery to gain the same soc.  
Since the added energy that a battery requires to gain a certain soc depends on multiple factors, we will try to capture as many factors as possible.  
To estimate the SoH all we really need to estimate is the energy required by a 100%SoH battery to gain a certain soc based on the charging point's factors.  
Then we just need to divide the energy required by the battery to gain a certain soc by the estimated energy required by a 100%SoH battery to gain the same soc in the same conditions.  

### Charging points SoH

#### Data extraction

In [ ]:
def compute_first_charge_soc(tss:DF) -> DF:
    tss["first_charge_soc"] = (
        tss
        .groupby(["vehicle_id", "trimmed_in_charge_idx"])
        ["soc"]
        .transform("first")
    )
    tss["first_charge_soc"] = tss["first_charge_soc"].where(tss["trimmed_in_charge"], pd.NA)
    return tss

charging_points:DF = (
    tss
    .pipe(compute_first_charge_soc)
    .query("trimmed_in_charge")
    .groupby(["vehicle_id", "trimmed_in_charge_idx", "soc"])
    .agg(
        energy_added_at_start=pd.NamedAgg(column="cum_energy_added", aggfunc="first"),
        energy_added_at_end=pd.NamedAgg(column="cum_energy_added", aggfunc="last"),
        energy_added=pd.NamedAgg(column="cum_energy_added", aggfunc=series_start_end_diff),
        ac_mode_mean=pd.NamedAgg(column="charging_ac_mode", aggfunc="mean"),
        dc_mode_mean=pd.NamedAgg(column="charging_dc_mode", aggfunc="mean"),
        current=pd.NamedAgg(column="ffilled_charging_current", aggfunc="median"),
        voltage=pd.NamedAgg(column="ffilled_charging_voltage", aggfunc="median"),
        estimated_range=pd.NamedAgg(column="ffilled_estimated_range", aggfunc="median"),
        time_remaining_for_charge=pd.NamedAgg(column="ffilled_time_remaining_for_charge", aggfunc="median"),
        model=pd.NamedAgg(column="vehicle_model", aggfunc="first"),
        first_charge_soc=pd.NamedAgg(column="first_charge_soc", aggfunc="first"),
        duration=pd.NamedAgg(column="date", aggfunc=series_start_end_diff),
    )
    .reset_index()
    .eval("energy_added=energy_added_at_end - energy_added_at_start")
    .eval("soc_added = soc - first_charge_soc")
    .eval("power = current * voltage")
    .eval("in_ac = ac_mode_mean > 0.3")
    .eval("in_dc = dc_mode_mean > 0.3")
    .eval("power = current * voltage")
    .eval("sec_duration = duration.dt.total_seconds()")
)

charging_points

#### EDA

In [ ]:
(
    charging_points
    .corr(numeric_only=True)
    .abs()
    .sort_values(by="energy_added", ascending=False)
    .loc[:, "energy_added"]
    .iloc[1:]
)

In [ ]:
plt_3d_df(
    (
        charging_points
        .query("energy_added > 0")
        #.query("vehicle_id == '666112423' | vehicle_id == '-465300286'")
        #.query("voltage > 300 & current.between(4700, 4900)")
        .dropna(subset=["voltage", "current", "energy_added"])
    ),
    x='power',
    y="soc_added",
    z="energy_added",
    color="vehicle_id",
    opacity=0.25,
    size=3,
    width=1500,
    height=1000,
    log_z=True,
    log_y=True,
)

Unfortunatly, there doesn't seem to be a strong correlation between the charging points' energy added and its factors.  

In [ ]:
px.histogram(
    charging_points,
    x="soc_added",
)